<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [1]:
import numpy as np
import random

In [2]:
class RedNeuronal:
    
    # num_entrada: dimension de los vectores de entrada, en iris: 4
    # tam_capas: lista de cantidad de neuronas por cada capa interna, en iris: [32, 16]
    # num_salida: dimension de los vectores de salida, en iris: 3
    # act_inter: función de activación de capas internas: "sigmoid" / "relu" / "arctan" / "softmax"
    # cost_func: función de costo: "crossentropy" / "mse" 
    def __init__(self, num_entrada, tam_capas, num_salida, act_inter, cost_func):
        self.num_entrada = num_entrada
        self.tam_capas = tam_capas
        self.num_salida = num_salida
        self.capas = []
        # iniciar todas las capas inicial - ocultas - final
        self.capas.append(CapaNeuronal(num_entrada, act_inter))
        for i in range(len(tam_capas)):
            self.capas.append(CapaNeuronal(tam_capas[i], act_inter, self.capas[i].tam_capa))
        self.capas.append(CapaNeuronal(num_salida, "softmax", self.capas[len(self.capas)-1].tam_capa))
        self.cant_capas = len(self.capas)
        if cost_func == "crossentropy":
            self.cost = self.crossentropy
            self.cost_derivative = self.crossentropy_derivative
        elif cost_func == "mse":
            self.cost = self.mse
            self.cost_derivative = self.mse_derivative
        
    def crossentropy(self, prediccion, esperado, epsilon=1e-12): #El epsilon evita log(0)
        prediccion = np.clip(prediccion, epsilon, 1. - epsilon)
        N = prediccion.shape[0]
        ce = -np.sum(np.sum(esperado*np.log(prediccion)))/N
        return ce
    
    def crossentropy_derivative(self, prediccion, esperado):
        return (esperado-1/prediccion+(1+esperado)*(1/(1-prediccion)))  
    
    def mse(self, prediccion, esperado):
        return np.dot(prediccion-esperado, prediccion-esperado)
    
    def mse_derivative(self, prediccion, esperado):
        return prediccion-esperado
    
    # train(entrada, salida, ciclos, tasa):
    # entrada: lista de datos de entrada
    # salida: lista de vectores de salida tipo onehot
    # ciclos: cantidad de epochs
    # tasa: tasa de aprendizaje 
    # retorna loss y accuracy
    def train(self, entrada, salida, ciclos, tasa, verbose = False):
        last = self.cant_capas-1 # indice capa de salida
        result = zip(entrada,salida)
        resultList = list(result)
        loss = []
        accuracy = []
        
        for ciclo in range(ciclos):
            # Comienza 1 epoch
            random.shuffle(resultList)
            errores = []
            acertados = 0
            for input_v, output in resultList:
                #Poblar capa inicial
                self.capas[0].set_capa(input_v, input_v) 
                
                # Feed forward 
                for j in range(1, len(self.capas)):
                    self.capas[j].feed_forward(self.capas[j-1])
                
                # evaluar error final (delta función costo)
                prediccion = self.capas[last].get_activaciones()
                error_epoch = self.cost(self.capas[last].get_activaciones(), output)
                errores.append(error_epoch)
                
                if (np.argmax(prediccion) == np.argmax(output)):
                    acertados += 1
                
                #Backward pass capa final
                ce_derivate = self.cost_derivative(prediccion, np.transpose(output))
                delta = ce_derivate * self.capas[last].squash_derivative(self.capas[last].get_vector_z())
                
                self.capas[last].actualizar_bias(delta, tasa)
                self.capas[last].actualizar_pesos(np.outer(self.capas[last-1].get_activaciones(), delta), tasa)

                #Backward pass general
                for l in range(2, self.cant_capas):
                    z = self.capas[-l].get_vector_z()
                    act_prime = self.capas[-l].squash_derivative(z)
                    delta = np.dot(self.capas[-l+1].get_pesos(), delta.transpose()) * act_prime
                    self.capas[-l].actualizar_bias(delta, tasa)
                    self.capas[-l].actualizar_pesos(np.outer(self.capas[-l-1].get_activaciones(),delta), tasa)
                
            loss.append(np.mean(errores))
            accuracy.append(acertados/len(resultList))
            if verbose:
                print("\tError del epoch "+str(ciclo)+": loss", str(np.round(np.mean(errores),5)),"- acc", acertados/len(resultList))
        print("Resultado de training set de", ciclos,"epochs: loss", np.mean(errores),"- acc:", acertados/len(resultList))
        return loss, accuracy

    # test(entrada, salida):
    # entrada: lista de datos de entrada de test
    # salida: Lista de datos esperados de test, en formato onehot vector
    # retorna loss y accuracy
    def test(self, entrada, salida):
        # repetir para todas 
        last = self.cant_capas-1 # indice capa de salida
        result = zip(entrada,salida)
        resultList = list(result)
        errores = []
        acertados = 0
        for input_v, output in resultList:
            self.capas[0].set_capa(input_v, input_v) #Poblar capa inicial
            # Feed forward 
            for j in range(1, len(self.capas)):
                self.capas[j].feed_forward(self.capas[j-1])

            
            prediccion = self.capas[last].get_activaciones()
            
            if (np.argmax(prediccion) == np.argmax(output)):
                    acertados += 1
                    
            # evaluar error final (delta función costo)
            error_epoch = self.crossentropy(prediccion, output)
            errores.append(error_epoch)
        
        print("Error del test:", str(np.round(np.mean(errores),7)))
        return np.mean(errores), acertados/len(resultList)
    
    def predict(self, entrada, categorias):
        last = self.cant_capas-1
        self.capas[0].set_capa(entrada, entrada)
        
        for j in range(1, len(self.capas)):
                self.capas[j].feed_forward(self.capas[j-1])
        
        prediccion = self.capas[last].get_activaciones()
        
        return categorias[np.argmax(prediccion)]

In [3]:
class CapaNeuronal:
    def __init__(self, tam_capa, tipo, tam_anterior = 0):
        self.tam_capa = tam_capa
        self.tam_anterior = tam_anterior
        self.bias = 1
        self.neuronas = [Neurona(tam_anterior) for i in range(tam_capa)] # lista de neuronas de la capa
        if tipo == "sigmoid":
            self.squash = self.sigmoid
            self.squash_derivative = self.sigmoid_derivative
        elif tipo == "softmax":
            self.squash = self.softmax
            self.squash_derivative = self.softmax_derivative
        elif tipo == "arctan":
            self.squash = self.arctan
            self.squash_derivative = self.arctan_derivative
        elif tipo == "relu":
            self.squash = self.relu
            self.squash_derivative = self.relu_derivative
       
    def sigmoid(self, x):
        return np.array([np.where(i > 0, 1. / (1. + np.exp(-i)), np.exp(i) / (np.exp(i) + np.exp(0))) for i in x])
    
    def sigmoid_derivative(self, x):
        return np.multiply(self.sigmoid(x),(1-self.sigmoid(x)))
    
    def arctan(self, x):
        return np.arctan(x)
    
    def arctan_derivative(self, x):
        return np.power(np.cos(x),2)
        
    def relu(self, x):
        x= np.array(x)
        return np.maximum(x, 0, x)
    
    def relu_derivative(self, x):
        x = np.array(x)
        return (x > 0).astype(int)
    
    def softmax(self, x):
        x = np.clip(x, -350, 350)
        return np.exp(x) / np.sum(np.exp(x), axis=0)
    
    def softmax_derivative(self, x):
        x = np.clip(x, -350, 350)
        res = []
        for i in x:
            res.append(np.exp(i)*(np.sum(np.exp(x), axis=0)-np.exp(i))/(np.sum(np.exp(x), axis=0)**2))
        #print(x)
        return np.array(res)
    
    def get_activaciones(self):
        return np.array([neurona.activacion for neurona in self.neuronas])
    
    def get_vector_z(self):
        return [neurona.z for neurona in self.neuronas]
    
    
    def feed_forward(self, capa_anterior):
        pre_squash = []
        activaciones = capa_anterior.get_activaciones()
        vector_z = capa_anterior.get_vector_z()
        for neurona in self.neuronas:
            pesos = neurona.pesos
            pre_squash.append(np.dot(pesos, activaciones) + neurona.bias)
        self.set_capa(self.squash(pre_squash), pre_squash)
        
   
    def get_pesos(self):
        return np.transpose(np.array([n.pesos for n in self.neuronas]))

    def actualizar_pesos(self, matriz, rate):
        pesos = np.transpose(np.array([n.pesos for n in self.neuronas]))
        pesos -= rate*matriz #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].pesos = pesos[:,i]
    
    def actualizar_bias(self, vector_b, rate):
        biases = np.array([n.bias for n in self.neuronas])
        biases -= rate*vector_b #SE PUSO SUMA PORQUE BAJABA EL ERROR DEL EPOCH
        for i in range(len(self.neuronas)):
            self.neuronas[i].bias = biases[i]
    
    #Funcion para poblar datos de entrada a la red.
    def set_capa(self, activaciones, vector_z):
        for i in range(len(self.neuronas)):
            self.neuronas[i].set_activacion(activaciones[i], vector_z[i])
            
    def mostrar_capa(self):
        for neurona in self.neuronas:
            print(neurona, end="\t")
        print("\n-----------------------------")

        
class Neurona:
    def __init__(self, tam_capa_anterior):
        self.pesos = np.random.rand(tam_capa_anterior) # genera una lista de numeros aleatorios de tamaño "tam_capa_anterior"
        self.bias = np.random.rand()
        
    def set_activacion(self, valor_a, valor_z):
        self.activacion = valor_a
        self.z = valor_z
        
    def __str__(self):
        return "Activación: "+str(self.activacion)

### ¡Es hora de probar!

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import keras
x, y = load_iris(return_X_y=True)
scaler = StandardScaler().fit(x)
x = scaler.transform(x) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
y_onehot_train = keras.utils.to_categorical(y_train)
y_onehot_test = keras.utils.to_categorical(y_test)
tags = ['setosa', 'versicolor', 'verginica']

# Instanciar red neuronal
nn = RedNeuronal(4, [32, 16], 3, "sigmoid", "mse")

Using TensorFlow backend.


In [5]:
#Entrenar la red
loss, accuracy = nn.train(x_train, y_onehot_train, 100, 0.1, verbose = True)

#Testear
print("\nTESTING")
nn.test(x_test, y_onehot_test)

	Error del epoch 0: loss 0.72314 - acc 0.33
	Error del epoch 1: loss 0.69996 - acc 0.41
	Error del epoch 2: loss 0.72263 - acc 0.28
	Error del epoch 3: loss 0.68812 - acc 0.37
	Error del epoch 4: loss 0.72664 - acc 0.25
	Error del epoch 5: loss 0.68842 - acc 0.38
	Error del epoch 6: loss 0.71495 - acc 0.29
	Error del epoch 7: loss 0.71306 - acc 0.3
	Error del epoch 8: loss 0.70904 - acc 0.3
	Error del epoch 9: loss 0.716 - acc 0.28
	Error del epoch 10: loss 0.7228 - acc 0.32
	Error del epoch 11: loss 0.7131 - acc 0.3
	Error del epoch 12: loss 0.71384 - acc 0.29
	Error del epoch 13: loss 0.69624 - acc 0.39
	Error del epoch 14: loss 0.68315 - acc 0.45
	Error del epoch 15: loss 0.70734 - acc 0.32
	Error del epoch 16: loss 0.71976 - acc 0.29
	Error del epoch 17: loss 0.70253 - acc 0.35
	Error del epoch 18: loss 0.70772 - acc 0.38
	Error del epoch 19: loss 0.70645 - acc 0.3
	Error del epoch 20: loss 0.69515 - acc 0.34
	Error del epoch 21: loss 0.67255 - acc 0.43
	Error del epoch 22: loss 0.

(0.11737933106230444, 0.92)

In [6]:
x_predict = x_test[0]

print(x_predict, y_onehot_test[0])
nn.predict(x_predict, tags)

[ 0.31099753 -0.58776353  0.53529583  0.00175297] [ 0.  1.  0.]


'versicolor'

In [7]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.plot(range(100), accuracy, label="Accuracy")
plt.plot(range(100), loss, label="Loss")
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

## 1. Referencias


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4650487614017582046
]


### Pregunta 2

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(3, activation="softmax"))
model.compile(optimizer=SGD(lr=0.1),loss="mse", metrics=["accuracy"])

In [10]:
model.fit(x_train, y_onehot_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
100/100 [==============================] - 0s 2ms/step - loss: 0.2366 - acc: 0.3100
Epoch 2/100
100/100 [==============================] - 0s 662us/step - loss: 0.2173 - acc: 0.3800
Epoch 3/100
100/100 [==============================] - 0s 667us/step - loss: 0.2073 - acc: 0.5500
Epoch 4/100
100/100 [==============================] - 0s 662us/step - loss: 0.1946 - acc: 0.6500
Epoch 5/100
100/100 [==============================] - 0s 672us/step - loss: 0.1782 - acc: 0.7100
Epoch 6/100
100/100 [==============================] - 0s 657us/step - loss: 0.1589 - acc: 0.6900
Epoch 7/100
100/100 [==============================] - 0s 647us/step - loss: 0.1440 - acc: 0.7300
Epoch 8/100
100/100 [==============================] - 0s 652us/step - loss: 0.1276 - acc: 0.7600
Epoch 9/100
100/100 [==============================] - 0s 652us/step - loss: 0.1191 - acc: 0.8500
Epoch 10/100
100/100 [==============================] - 0s 657us/step - loss: 0.1034 - acc: 0.8400
Epoch 11/100
100/100 

100/100 [==============================] - 0s 652us/step - loss: 0.0154 - acc: 0.9600
Epoch 84/100
100/100 [==============================] - 0s 657us/step - loss: 0.0152 - acc: 0.9700
Epoch 85/100
100/100 [==============================] - 0s 652us/step - loss: 0.0137 - acc: 0.9900
Epoch 86/100
100/100 [==============================] - 0s 652us/step - loss: 0.0156 - acc: 0.9600
Epoch 87/100
100/100 [==============================] - 0s 647us/step - loss: 0.0136 - acc: 0.9900
Epoch 88/100
100/100 [==============================] - 0s 657us/step - loss: 0.0147 - acc: 0.9800
Epoch 89/100
100/100 [==============================] - 0s 652us/step - loss: 0.0135 - acc: 0.9800
Epoch 90/100
100/100 [==============================] - 0s 652us/step - loss: 0.0150 - acc: 0.9800
Epoch 91/100
100/100 [==============================] - 0s 652us/step - loss: 0.0141 - acc: 0.9700
Epoch 92/100
100/100 [==============================] - 0s 647us/step - loss: 0.0137 - acc: 0.9800
Epoch 93/100
100/100 [=